 <div align="center">

  <a href="https://www.mbari.org/" target="_blank">
    <img width="512", src="https://www.mbari.org/wp-content/uploads/2014/11/logo-mbari-3b.png"></a>


This notebook by <a href="https://mbari.org">MBARI</a> demonstrates how to train a YOLOv5 object detection model using freely available FathomNet data using AWS. See the <a href="https://docs.mbari.org/deepsea-ai/">full documentation</a> for information on the deepsea-ai module or check <a href="https://github.com/mbari-org/deepsea-ai/discussions">github</a> for questions.
    
Select the kernel **deepsea-ai-notebooks**
</div>

## Setup
Set the [installation instructions](https://docs.mbari.org/deepsea-ai/) for details on setting up your AWS account.

A few housekeeping items:

1. The packages installed require python3.7 or greater, so let's check the python version first, then install the additional packages used in this notebook
2. Verify your AWS account is working by listing your buckets

In [ ]:
!python --version

In [ ]:
!pip install -U -q fathomnet pillow
!pip install -U -q deepsea-ai

In [ ]:
!aws s3 ls

## Select concepts

Let's choose some concepts that are in the FathomNet database and put them in a list.  We are also going to put them into a file, one line per concept in alphabetical order which will be used during training.

In [ ]:
concepts = ["Rathbunaster californicus", "Holothuroidea", "Strongylocentrotus fragilis"]

In [ ]:
with open('mynames.txt', 'w+') as n:
  for c in sorted(concepts):
    n.write(c + '\r\n')

#This should be in alphabetical order
!cat mynames.txt

Let's call the `fathomnet.api.images.find_by_concept` function to query how many images are available for these concepts.

In [ ]:
from fathomnet.api import images

for c in concepts:
  available_images = images.find_by_concept(c)
  print(f'{c} images: {len(available_images)}')

## Download the images and format the labels 

This code will:
1. **Download** the images and bounding box annotations from FathomNet and save them to the ```trainingdata/``` folder.
2. **Format** the bounding boxes into a format compatible with the object detection model.  The YOLOv5 model uses a very simple format which is a text file, one line per annotation, e.g. ```
0 101 33 83 199
``` which corresponds to```<label index 1-based> <x> <y> <width> <height>
```

In [ ]:
import requests
from pathlib import Path
from progressbar import progressbar
from io import BytesIO
from fathomnet.api import images
from PIL import Image
from fathomnet.models import GeoImageConstraints

# Create a directory for the images and labels
data_dir = Path.cwd() / 'trainingdata'

image_dir = data_dir / 'images'
image_dir.mkdir(exist_ok=True, parents=True)

label_dir = data_dir / 'labels'
label_dir.mkdir(exist_ok=True, parents=True)


# Download each image and create a label file for training
image_paths = []

for c in concepts:
  
  # Constrain to only 50 images
  concept_constrained = GeoImageConstraints(concept=c, limit=50)
  concept_images = images.find(concept_constrained)

  print(f'Downloading {c} images: {len(concept_images)}')
    
  for image in progressbar(concept_images):
    # Export labels with the same name as the unique identifier appended with .txt per darknet format
    label = label_dir / f'{image.uuid}.txt'
    print(image.boundingBoxes)

    # Export to darknet format, which is one line per annotation <label index 1-based> <x> <y> <width> <height>
    with label.open("w+") as l:
      for b in image.boundingBoxes:
        # Only save concepts in our list
        if b.concept in concepts:
          l.write(f'{concepts.index(b.concept) + 1} {b.x} {b.y} {b.width} {b.height}')
    
    # # Format our image file name as the image UUID + .jpg
    image_path = image_dir / f'{image.uuid}.jpg'
    image_paths.append(image_path)
    if image_path.exists():  # Skip re-downloading images
      continue
    
    # # Download the image
    image_raw = requests.get(image.url, stream=True).raw
    pil_image = Image.open(image_raw)
    
    # Convert to RGB (ensures consistent colorspace)
    pil_image = pil_image.convert('RGB')

    # Save the image
    pil_image.save(image_path)


## Train

Now that we have some training data, let's train it for a few cycles.

Before we train it, we need to split the data into training, testing.  The data is split then compressed into images.tar.gz and labels.tar.gz files to reduce the data size.  

```
├── trainingdata
│   │   ├── images
│   │   │   └── image1.jpg
│   │   │   └── image2.jpg
│   │   ├── labels
│   │   │   └── image1.txt
│   │   │   └── image2.txt 
```

```
├── splitdata
│   │   ├── images.tar.gz
│   │   ├── labels.tar.gz
```

In [ ]:
!deepsea-ai split -i trainingdata -o splitdata

## Train the object detection model for a few epochs

The object detector model is the highly performant [YOLOv5 model](http://github.com/ultralytics/yolov5). This represents a good tradeoff between cost and performance. See [training](https://docs.mbari.org/deepsea-ai/commands/train/) for the full documentation.

To track cost, we will setup unique tags for our project, and pass that to the ``--config`` option.

In [ ]:
%%writefile deepseailab.txt
[aws]
sagemaker_arn =
yolov5_ecr = mbari/deepsea-yolov5:1.1.2
deepsort_ecr = mbari/deepsort-yolov5:latest
strongsort_ecr = mbari/strongsort-yolov5:latest
yolov5_model_s3 = s3://902005-public/models/yolov5x_mbay_benthic_model.tar.gz
deepsort_track_config_s3 = s3://902005-public/models/deep_sort_benthic.yaml
strongsort_track_config_s3 = s3://902005-public/models/strong_sort_benthic.yaml

[database]
site = http://deepsea-ai.shore.mbari.org
gql = %(site)/graphql

[tags]
organization = deepseailab
project_number = NA
stage = test
application = detection

## Create a unique bucket name

Please note that bucket names need to be globally unique.  Let's set the bucket name here with a randomly generated string to help with that. You can remove this and replace with your unique bucket name. 

In [ ]:
import uuid
uuid_short = str(uuid.uuid4())[:8]
uuid_short

## Train
We set the number of training iterations to **2** which is far too small to be useful, but we will test it first to make sure your AWS account works ok.  We will use the ml.p3.2xlarge instance which is at the time of this writing $3.06 USD per hour to use.

In [ ]:
!deepsea-ai train --model yolov5x --instance-type ml.p3.8xlarge \
--labels splitdata/labels.tar.gz \
--images splitdata/images.tar.gz \
--label-map mynames.txt \
--config deepseailab.txt \
--input-s3 f's3://{uuid_short}-deepseailab-benthic-training/' \
--output-s3 f's3://{uuid_short}-deepseailab-benthic-checkpoints/' \
--epochs 2 \
--batch-size 2